# Interoperability with C

Julia has a simple way to call C and Fortran functions in shared libraries, via the `ccall` function.

In [1]:
?ccall

search: ccall AbstractChannel



```
ccall((symbol, library) or function_pointer, ReturnType, (ArgumentType1, ...), ArgumentValue1, ...)
```

Call function in C-exported shared library, specified by `(function name, library)` tuple, where each component is a string or symbol.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression. Alternatively, `ccall` may also be used to call a function pointer, such as one returned by `dlsym`.

Each `ArgumentValue` to the `ccall` will be converted to the corresponding `ArgumentType`, by automatic insertion of calls to `unsafe_convert(ArgumentType, cconvert(ArgumentType, ArgumentValue))`. (See also the documentation for each of these functions for further details.) In most cases, this simply results in a call to `convert(ArgumentType, ArgumentValue)`.


We see that we must specify:

- the name of the function, as a Julia symbol or as a string
- and the name of the shared library where it lives;  these are given as an ordered pair (tuple)


- the return type of the function

- the argument types that the function accepts, as a tuple

- and the arguments themselves

A simple example is to call the clock function:

In [2]:
t = ccall( (:clock, "libc"), Int32, ())

4477982

In [3]:
typeof(t)

Int32

In [4]:
path = ccall( (:getenv, "libc"), Ptr{UInt8}, (Ptr{UInt8},), "PATH")

Ptr{UInt8} @0x00007f91066b4a85

Here, `Ptr` denotes a pointer to the given type.

In [5]:
unsafe_string(path) # ENV["PATH"]

"/Applications/Julia-0.5.app/Contents/Resources/julia/bin:/Applications/Julia-0.5.app/Contents/Resources/julia/bin:/usr/local/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin"

In [6]:
?unsafe_string

search: unsafe_string unsafe_store!



```
unsafe_string(p::Ptr{UInt8}, [length::Integer])
```

Copy a string from the address of a C-style (NUL-terminated) string encoded as UTF-8. (The pointer can be safely freed afterwards.) If `length` is specified (the length of the data in bytes), the string does not have to be NUL-terminated.

This function is labelled "unsafe" because it will crash if `p` is not a valid memory address to data of the requested length.

See also [`unsafe_wrap(String, p, [length])`](:func:`unsafe_wrap`), which takes a pointer and wraps a string object around it without making a copy.


C libraries tend to have large, repetitive, uniform API patterns and naming conventions.

Julia macros combined with `ccall` can make it easy to write wrapper interfaces.

Example: MPFR for `BigFloat`:

In [7]:
a = big"0.1"
b = big"0.3"
@which a + b

+(x::BigFloat, y::BigFloat) at mpfr.jl:205

In [8]:
@which sin(a)

sin(x::BigFloat) at mpfr.jl:610

Automated wrapper generation from headers is possible via the Clang.jl package: https://github.com/ihnorton/Clang.jl